# Learning MNIST dataset with Keras

Required Packages:
* SciPy
* NumPy
* Matplotlib
* Tensorflow or Theano

### Import Libraries

In [1]:
import numpy as np

# For reproducibility, use a static seed
np.random.seed(123)

# Keras model module
from keras.models import Sequential

# Keras core layers
from keras.layers import Dense, Dropout, Activation, Flatten

# Keras CNN layers
from keras.layers import Conv2D, MaxPooling2D

# Utilities
from keras.utils import np_utils

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Load image data from MNIST

In [2]:
# Load MNIST data
from keras.datasets import mnist

# Load pre-shuffled MNIST data into training and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print(X_train.shape)

(60000, 28, 28)


### Plot the first sample of X_train

In [3]:
from matplotlib import pyplot as plt
plt.imshow(X_train[0])

## Preprocess input data for Keras

Our image data is of the form:
> (n, width, height) 
> _where **n** is the number of images_.

Target Form for our data:
> We need our data to include **image depth** such that our data is of the form
> (n, **depth**, width, height)


In [4]:
X_train.shape
# (60000, 28, 28)
# - 60000 : images
# -    28 : width  (px)
# -    28 : height (px)

(60000, 28, 28)

In [5]:
# Shape of a single image
X_train[0].shape

(28, 28)

In [6]:
# pixel_value @ imgSet[img][row][col]
X_train[0][10][12]

253

##### Include image depth (where R,G,B = 3 deep)
- MNIST images have a DEPTH of 1

In [7]:
# Reshape input data
DEPTH = 1
# X_train = X_train.reshape(X_train.shape[0], DEPTH, 28, 28)
# X_test = X_test.reshape(X_test.shape[0], DEPTH, 28, 28)

X_train = X_train.reshape(X_train.shape[0], 28, 28, DEPTH)
X_test = X_test.reshape(X_test.shape[0], 28, 28, DEPTH)

In [8]:
print(X_train.shape)

(60000, 28, 28, 1)


#### Normalize pixel values

Current Form:
- int value (0 to 255)

Target Form:
- float value (0.0 to 1.0)

In [9]:
# Convert data type and normalize valuesPython

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

### Preprocess class labels for Keras

In [10]:
print(y_train.shape)

# (60000,)
# 1-dimensional dataset ==> 10-dimensional dataset

(60000,)


###### Values of y_train

In [11]:
y_train_values = set()
[y_train_values.add(x) for x in y_train]

# Show the set of values in y_train
y_train_values

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

###### Split the values into their own columns

Current Form:
- y_train is a 1-dimensional array of values ranging from 0 to 9
    - ( 0, 2, 9, 5, 1, ...)

Target Form:
- reshape y_train into a 10-dimensional array of classes where:
    - each column corresponds to the class
    - value at each column is 1 (True) or 0 (False)

In [12]:
# Reshape y_train so that it has 10 different classes
print(y_train[:10])

[5 0 4 1 9 2 1 3 1 4]


In [13]:
# create Y_train
Y_train =  np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

print(Y_train.shape)

# (60000, 10)

(60000, 10)


In [14]:
# In old form
y_train[0]

# 5

5

In [15]:
# In target (categorical) form
Y_train[0]

# array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

## Defining the Model Architecture

In [16]:
# Declare Sequential model 
model = Sequential()

### IMPORTANT: Kernel size & Sample (image) size

The input shape parameter should be the shape of **1 sample**.
In our case, since the image is 28 x 28, then the kernel will have the same shape.

#### Conv2D
usage:
> keras.layers.Conv2D(filters, kernel_size, strides=(1, 1), padding='valid', data_format=None, dilation_rate=(1, 1), activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)

In [17]:
# CNN input layer
# Conv2D(numConvFiltersToUse, (numRowsInConvFilter, numColumnsInConvFilter))
model.add(Conv2D(32, (3,3), input_shape=(28,28, 1), activation='relu'))
model.add(Conv2D(32, (3,3), activation='relu'))

#### MaxPooling2D
A way to reduce the number of parameters in our model by sliding a 2x2 across the previous layer and taking the max of the 4 values in the 2x2 filter.

In [18]:
model.add(MaxPooling2D(pool_size=(2,2)))

#### DropOut
A method for regularizing our model in order to prevent overfitting.

In [19]:
model.add(Dropout(0.25))

#### Flatten
It's important to note that the weights from the Convolution layers must be flattened (made 1-dimensional again) before we can pass them to the fully connected Dense layer.

In [20]:
model.add(Flatten())

#### Dense
For Dense layers, the first parameter is the output size of the layer. Keras automatically handles the connections between layers (Dense = fully-connected)

In [21]:
model.add(Dense(128, activation='relu'))

In [22]:
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

## Compile Model

Compile the model so that we train it. When we compile the model, we declare things such as the **loss function**, the **optimizer** (SGD, adam, ...), etc.

In [23]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

## Train
To fit the model, all we have to do is declare the batch size and number of epochs to train for, then pass in our **training** data.

In [24]:
X_train.shape

(60000, 28, 28, 1)

In [25]:
type(X_train)

numpy.ndarray

In [27]:
model.fit(X_train,
          Y_train, 
          batch_size=32,
          epochs=10, 
          verbose=1)

Epoch 1/10
60000/60000 [==============================] - 306s - loss: 0.1112 - acc: 0.9677   
Epoch 2/10
60000/60000 [==============================] - 302s - loss: 0.0807 - acc: 0.9776   
Epoch 3/10
60000/60000 [==============================] - 302s - loss: 0.0707 - acc: 0.9803   
Epoch 4/10
60000/60000 [==============================] - 302s - loss: 0.0646 - acc: 0.9816   
Epoch 5/10
60000/60000 [==============================] - 302s - loss: 0.0636 - acc: 0.9823   
Epoch 6/10
60000/60000 [==============================] - 302s - loss: 0.0631 - acc: 0.9827   
Epoch 7/10
60000/60000 [==============================] - 303s - loss: 0.0649 - acc: 0.9824   
Epoch 8/10
60000/60000 [==============================] - 303s - loss: 0.0654 - acc: 0.9826   
Epoch 9/10
60000/60000 [==============================] - 303s - loss: 0.0656 - acc: 0.9823   
Epoch 10/10
60000/60000 [==============================] - 303s - loss: 0.0664 - acc: 0.9824   
